<a href="https://colab.research.google.com/github/Calvince1/Colabs/blob/main/Fraud%20in%20Tax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TAX FRAUD DETECTION FOR UNDER DECLARATIONS USING UNSUPERVISED MACHINE LEARNING MODEL**

Taxation serves as an essential method for generating revenue, enabling governments to furnish essential goods and services. However, many people dont like paying taxes and put strategies to either avoid or reduce tax liability.

Some of th Strategies include:
1. Falsifying VAT input invoices
2. Double Claim of VAT inputs
3. Underdeclaration of Sales
4. Overclaim of Purchases & ID claims
5. Use of an inconsistent stock valuation method

**Research Objectives**
1. Establish the relationship between income tax turnover and sales volume declared
2. Develop a model to predict sales underdeclarations
3.

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import pylab as pl
import numpy as np
%matplotlib inline

In [3]:
df_turn = pd.read_csv("/content/drive/MyDrive/New folder/turnover dataset_Y.csv")
df_turn.head()

,Pin No,TP Name,Station Name,Economic Activity,it_trp_from_dt,it_trp_to_dt,it_acc_year,vat_from_month,vat_to_month,period_year,acc_year,vat_returns_filed,Vat Sales Volume,VAT payments,Income Tax Turnover,Income tax Pa Payments,it_vat_variance
0,P000591112Z,The Limuru Tea Company Limited,East of Nairobi,"AGRICULTURE, FORESTRY AND FISHING",01/01/2021,31/12/2021,2021,2021-01,2021-12,2021,2021,12,"82,970,917","387,655",0,"1,580,911","-82,970,917"
1,P000591113A,KAKUZI PLC,Large Tax payer,"AGRICULTURE, FORESTRY AND FISHING",01/01/2021,31/12/2021,2021,2021-01,2021-12,2021,2021,12,"3,321,902,056",0,"386,580","157,722,649","-3,321,515,476"
2,P000591126I,Kaisugu Limited,Large Tax payer,"AGRICULTURE, FORESTRY AND FISHING",01/01/2021,31/12/2021,2021,2021-01,2021-12,2021,2021,12,"1,054,528,528","1,045,323",0,"19,260,029","-1,054,528,528"
3,P000591479A,Modern Ways Limited,MTO,"AGRICULTURE, FORESTRY AND FISHING",01/01/2021,31/12/2021,2021,2021-01,2021-12,2021,2021,12,"414,777,717",0,"412,814,067",0,"-1,963,650"
4,P000591772W,BAYER EAST AFRICA LIMITED,Large Tax payer,"AGRICULTURE, FORESTRY AND FISHING",01/01/2021,31/12/2021,2021,2021-01,2021-12,2021,2021,12,"4,235,759,850",0,"4,772,699,000","203,229,306","536,939,150"


In [4]:
df_turn.columns

Index(['Pin No', 'TP Name', 'Station Name', 'Economic Activity',
       'it_trp_from_dt', 'it_trp_to_dt', 'it_acc_year', 'vat_from_month',
       'vat_to_month', 'period_year', 'acc_year', 'vat_returns_filed',
       'Vat Sales Volume', 'VAT payments', 'Income Tax Turnover',
       'Income tax Pa Payments', 'it_vat_variance'],
      dtype='object')

In [5]:
#selecting some useful features
data = df_turn[['Station Name', 'Economic Activity','Vat Sales Volume', 'VAT payments', 'Income Tax Turnover',
       'Income tax Pa Payments', 'it_vat_variance']]
data.head()

,Station Name,Economic Activity,Vat Sales Volume,VAT payments,Income Tax Turnover,Income tax Pa Payments,it_vat_variance
0,East of Nairobi,"AGRICULTURE, FORESTRY AND FISHING","82,970,917","387,655",0,"1,580,911","-82,970,917"
1,Large Tax payer,"AGRICULTURE, FORESTRY AND FISHING","3,321,902,056",0,"386,580","157,722,649","-3,321,515,476"
2,Large Tax payer,"AGRICULTURE, FORESTRY AND FISHING","1,054,528,528","1,045,323",0,"19,260,029","-1,054,528,528"
3,MTO,"AGRICULTURE, FORESTRY AND FISHING","414,777,717",0,"412,814,067",0,"-1,963,650"
4,Large Tax payer,"AGRICULTURE, FORESTRY AND FISHING","4,235,759,850",0,"4,772,699,000","203,229,306","536,939,150"


In [6]:
#Checking Data types
data.dtypes

Station Name              object
Economic Activity         object
Vat Sales Volume          object
VAT payments              object
Income Tax Turnover       object
Income tax Pa Payments    object
it_vat_variance           object
dtype: object

In [7]:
#Converting select columns to int
data['Income Tax Turnover'] = data['Income Tax Turnover'].str.replace(',', '').astype(int)
data['it_vat_variance'] = data['it_vat_variance'].str.replace(',', '').astype(int)
data['VAT payments'] = data['VAT payments'].str.replace(',', '').astype(int)
data['Income tax Pa Payments'] = data['Income tax Pa Payments'].str.replace(',', '').astype(int)
data['Vat Sales Volume'] = data['Vat Sales Volume'].str.replace(',', '').astype(int)
data.dtypes

<ipython-input-7-b2e91040525a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Income Tax Turnover'] = data['Income Tax Turnover'].str.replace(',', '').astype(int)
<ipython-input-7-b2e91040525a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['it_vat_variance'] = data['it_vat_variance'].str.replace(',', '').astype(int)
<ipython-input-7-b2e91040525a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

Station Name              object
Economic Activity         object
Vat Sales Volume           int64
VAT payments               int64
Income Tax Turnover        int64
Income tax Pa Payments     int64
it_vat_variance            int64
dtype: object